## Initializing Grid

In [53]:
#Grid Class
class Grid:
  #Initalizing grid with number of rows and number of columns
  def __init__(self,rows,cols,blocked):
    self.rows = rows
    self.cols = cols
    self.Uniblocked = blocked
    self.currblocked = []
  #Returns neighbors of given cell if they are not blocked in the current state
  def neighbors(self,id):
    (x,y) = id
    neighbors =  (x+1,y),(x-1,y),(x,y-1),(x,y+1)
    res = filter(self.inbound,neighbors)
    res = filter(self.passable,res)
    return res
  #checks if the cells are not crossing the limit of rows and columns
  def inbound(self,id):
    (x,y) = id
    return (0 <= x < self.rows) and (0 <= y < self.cols)
  #checks if cells are not blocked in the current state
  def passable(self,id):
    (x,y) = id
    return id not in self.currblocked
  #cost to move from one cell to another
  def cost(self,f,t):
    return 1

# Heap which sorts by min value of G

In [54]:
class PriorityQueueMinG:
  def __init__(self):
    self.heapq = [0]
    self.priority = {}

  #inserts node into heap with priority given to min F value and min g-value
  def put(self,x,nodepriority):
    self.heapq.append(x)
    self.priority[x] = nodepriority
    index = len(self.heapq)-1
    #loop till you find min f-value and g-value
    while((index//2)>=1 and self.priority[self.heapq[index]][0]<=self.priority[self.heapq[index//2]][0]):
      #if F-values are equal check g-values
      if self.priority[self.heapq[index]][0]==self.priority[self.heapq[index//2]][0]:
        #if g-value of parent is smaller then child - swap
        if self.priority[self.heapq[index]][1]<self.priority[self.heapq[index//2]][1]:#breaking ties using smaller g values
          self.swap(index,index//2)    
      #if f-value of parent is smaller - swap
      else:
        self.swap(index,index//2)
      index = index//2
    return self.heapq

  #check if heap is empty
  def empty(self):
    return len(self.heapq) == 1

  #Returns minimum element
  def peek(self):
    return self.heapq[1]


  def get(self):
    max_ele = self.heapq[1]
    self.heapq[1] = self.heapq[-1]
    self.heapq = self.heapq[:-1]
    index = 1
    left_child = 2*index
    right_child = 2*index+1
    cond1 = right_child<len(self.heapq)
    cond2 = left_child<len(self.heapq)
    while cond1 or cond2:
      
      #both left_child and right_child
      if cond1:
        #parent F < left_child F and right_child F - done
        if self.priority[self.heapq[index]][0]<self.priority[self.heapq[left_child]][0] and self.priority[self.heapq[index]][0]<self.priority[self.heapq[right_child]][0]:
          return max_ele
        
        #parent F >left_child and right_child - check which is smaller
        elif self.priority[self.heapq[index]][0]>self.priority[self.heapq[left_child]][0] and self.priority[self.heapq[index]][0]>self.priority[self.heapq[right_child]][0]:
        #check which child has smaller F
            #left child has smaller F - swap with left child
            if self.priority[self.heapq[left_child]][0]< self.priority[self.heapq[right_child]][0]:
                self.swap(index,left_child)
                index = left_child
            #right child has smaller F -  swap with right_child        
            elif self.priority[self.heapq[left_child]][0]> self.priority[self.heapq[right_child]][0]:   
                self.swap(index,right_child)
                index = right_child
            #left child F = right child F - check g values
            else:
                #left_child G  < right child G - swap with left_child
                if self.priority[self.heapq[left_child]][1]<self.priority[self.heapq[right_child]][1]:
                    self.swap(index,left_child)
                    index = left_child
                #right_child G <= left child G - swap with right child
                else:
                    self.swap(index,right_child)
                    index = right_child

        #parent F == left_child F and parent F == right_child F - check g values
        elif self.priority[self.heapq[index]][0] == self.priority[self.heapq[left_child]][0] and self.priority[self.heapq[index]][0] == self.priority[self.heapq[right_child]][0]:
            #g_value of parent is smaller than left_child and right_child - done
            if self.priority[self.heapq[index]][1]< self.priority[self.heapq[left_child]][1] and self.priority[self.heapq[index]][1]< self.priority[self.heapq[right_child]][1]:
                return max_ele
            #g_value of parent is equal to left and right child - done
            elif self.priority[self.heapq[index]][1] == self.priority[self.heapq[left_child]][1] and self.priority[self.heapq[index]][1] == self.priority[self.heapq[right_child]][1]:
                return max_ele
            #g_value of parent is greater than left child and right child - check which is smaller
            elif self.priority[self.heapq[index]][1]> self.priority[self.heapq[left_child]][1] and self.priority[self.heapq[index]][1]>self.priority[self.heapq[right_child]][1]:
                #g_value of left < g_value of right - swap parent with left
                if self.priority[self.heapq[left_child]][1]< self.priority[self.heapq[right_child]][1]:
                    self.swap(index,left_child)
                    index = left_child
                #g_value of right <= left_child - swap with right child
                else:
                    self.swap(index,right_child)
                    index = right_child
            
            #g_value of parent is greater than left_child but not right_child - swap with left
            elif self.priority[self.heapq[left_child]][1]< self.priority[self.heapq[index]][1]:
                self.swap(left_child,index)
                index = left_child
            #g_value of parent is greater than right_child but not left_child - swap with right
            elif self.priority[self.heapq[right_child]][1]< self.priority[self.heapq[index]][1]:
                self.swap(right_child,index)
                index = right_child
            #g_value of parent is equal to one of the child but other child is greater than parent-done
            else:
                return max_ele
        
        #parent F if greater than left child but not right child - swap with left
        elif self.priority[self.heapq[left_child]][0]< self.priority[self.heapq[index]][0]:
            self.swap(left_child,index)
            index = left_child
        
        #parent F is greater than right child but not left child - swap with right
        elif self.priority[self.heapq[right_child]][0]< self.priority[self.heapq[index]][0]:
            self.swap(right_child,index)
            index = right_child  
        
        #F-value of parent is equal to one of the children and other child is greater than parent - find equal and check g-values  
        else: 
            #F-value of left = F-value of parent - check G
            if self.priority[self.heapq[left_child]][0] == self.priority[self.heapq[index]][0]:
                #left child has smaller G than parent - swap
                if self.priority[self.heapq[left_child]][1]<self.priority[self.heapq[index]][1]:
                    self.swap(left_child,index)
                    index = left_child
                #left_child G >= parent G
                else:
                    return max_ele
            #F-value of right = F-value of parent - check G
            elif self.priority[self.heapq[right_child]][0] == self.priority[self.heapq[index]][0]:
                #right child has smaller G than parent - swap
                if self.priority[self.heapq[right_child]][1]<self.priority[self.heapq[index]][1]:
                    self.swap(right_child,index)
                    index = right_child
                #right_child G >= parent G
                else:
                    return max_ele
        
      else:#only left child
        
        #parent F < left_child F done
        if self.priority[self.heapq[index]][0]<self.priority[self.heapq[left_child]][0]:
          return max_ele

        #parent F > left_child F need to swap
        elif self.priority[self.heapq[index]][0]>self.priority[self.heapq[left_child]][0]:
            self.swap(left_child,index)
            index = left_child

        #parent F = left_child F check G
        elif self.priority[self.heapq[index]][0] == self.priority[self.heapq[left_child]][0]:
        #parent G <= left_child G done
          if self.priority[self.heapq[index]][1] <= self.priority[self.heapq[left_child]][1]:
            return max_ele
          #parent G > left_child G need to swap
          else: 
            self.swap(index,left_child)
            index = left_child
      
      #incerement child indexes
      left_child = 2*index
      right_child = 2*index+1
      cond1 = right_child<len(self.heapq)
      cond2 = left_child<len(self.heapq)
     

    return max_ele

  def swap(self,child,parent):
    self.heapq[child],self.heapq[parent] = self.heapq[parent],self.heapq[child]

# Heap which sorts by max Value of G

In [55]:
class PriorityQueueMaxG:
  def __init__(self):
    self.heapq = [0]
    self.priority = {}

  #inserts node into heap with priority given to min F value and max g-value
  def put(self,x,nodepriority):
    self.heapq.append(x)
    self.priority[x] = nodepriority
    index = len(self.heapq)-1
    #loop till you find min f-value and max g-value
    while((index//2)>=1 and self.priority[self.heapq[index]][0]<=self.priority[self.heapq[index//2]][0]):
      #if F-values are equal check g-values
      if self.priority[self.heapq[index]][0]==self.priority[self.heapq[index//2]][0]:
        #if g-value of parent is larger then child - swap
        if self.priority[self.heapq[index]][1]>self.priority[self.heapq[index//2]][1]:
          self.swap(index,index//2)    
      #if f-value of parent is smaller - swap
      else:
        self.swap(index,index//2)
      index = index//2
    return self.heapq

  #check if heap is empty
  def empty(self):
    return len(self.heapq) == 1

  #Returns minimum element
  def peek(self):
    return self.heapq[1]


  def get(self):
    max_ele = self.heapq[1]
    self.heapq[1] = self.heapq[-1]
    self.heapq = self.heapq[:-1]
    index = 1
    left_child = 2*index
    right_child = 2*index+1
    cond1 = right_child<len(self.heapq)
    cond2 = left_child<len(self.heapq)
    while cond1 or cond2:
      
      #both left_child and right_child
      if cond1:
        #parent F < left_child F and right_child F - done
        if self.priority[self.heapq[index]][0]<self.priority[self.heapq[left_child]][0] and self.priority[self.heapq[index]][0]<self.priority[self.heapq[right_child]][0]:
          return max_ele
        
        #parent F >left_child and right_child - check which is smaller
        elif self.priority[self.heapq[index]][0]>self.priority[self.heapq[left_child]][0] and self.priority[self.heapq[index]][0]>self.priority[self.heapq[right_child]][0]:
        #check which child has smaller F
            #left child has smaller F - swap with left child
            if self.priority[self.heapq[left_child]][0]< self.priority[self.heapq[right_child]][0]:
                self.swap(index,left_child)
                index = left_child
            #right child has smaller F -  swap with right_child        
            elif self.priority[self.heapq[left_child]][0]> self.priority[self.heapq[right_child]][0]:   
                self.swap(index,right_child)
                index = right_child
            #left child F = right child F - check g values
            else:
                #left_child G  > right child G - swap with left_child
                if self.priority[self.heapq[left_child]][1]>self.priority[self.heapq[right_child]][1]:
                    self.swap(index,left_child)
                    index = left_child
                #right_child G >= left child G - swap with right child
                else:
                    self.swap(index,right_child)
                    index = right_child

        #parent F == left_child F and parent F == right_child F - check g values
        elif self.priority[self.heapq[index]][0] == self.priority[self.heapq[left_child]][0] and self.priority[self.heapq[index]][0] == self.priority[self.heapq[right_child]][0]:
            #g_value of parent is larger than left_child and right_child - done
            if self.priority[self.heapq[index]][1]> self.priority[self.heapq[left_child]][1] and self.priority[self.heapq[index]][1]> self.priority[self.heapq[right_child]][1]:
                return max_ele
            #g_value of parent is equal to left and right child - done
            elif self.priority[self.heapq[index]][1] == self.priority[self.heapq[left_child]][1] and self.priority[self.heapq[index]][1] == self.priority[self.heapq[right_child]][1]:
                return max_ele
            #g_value of parent is smaller than left child and right child - check which is larger
            elif self.priority[self.heapq[index]][1]< self.priority[self.heapq[left_child]][1] and self.priority[self.heapq[index]][1]<self.priority[self.heapq[right_child]][1]:
                #g_value of left > g_value of right - swap parent with left
                if self.priority[self.heapq[left_child]][1]> self.priority[self.heapq[right_child]][1]:
                    self.swap(index,left_child)
                    index = left_child
                #g_value of right <= left_child - swap with right child
                else:
                    self.swap(index,right_child)
                    index = right_child
            
            #g_value of parent is smaller than left_child but not right_child - swap with left
            elif self.priority[self.heapq[left_child]][1]> self.priority[self.heapq[index]][1]:
                self.swap(left_child,index)
                index = left_child
            #g_value of parent is smaller than right_child but not left_child - swap with right
            elif self.priority[self.heapq[right_child]][1]> self.priority[self.heapq[index]][1]:
                self.swap(right_child,index)
                index = right_child
            #g_value of parent is equal to one of the child but other child is greater than parent-done
            else:
                return max_ele
        
        #parent F if greater than left child but not right child - swap with left
        elif self.priority[self.heapq[left_child]][0]< self.priority[self.heapq[index]][0]:
            self.swap(left_child,index)
            index = left_child
        
        #parent F is greater than right child but not left child - swap with right
        elif self.priority[self.heapq[right_child]][0]< self.priority[self.heapq[index]][0]:
            self.swap(right_child,index)
            index = right_child  
        
        #F-value of parent is equal to one of the children and other child is greater than parent - find equal and check g-values  
        else: 
            #F-value of left = F-value of parent - check G
            if self.priority[self.heapq[left_child]][0] == self.priority[self.heapq[index]][0]:
                #left child has larger G than parent - swap
                if self.priority[self.heapq[left_child]][1]>self.priority[self.heapq[index]][1]:
                    self.swap(left_child,index)
                    index = left_child
                #left_child G <= parent G
                else:
                    return max_ele
            #F-value of right = F-value of parent - check G
            elif self.priority[self.heapq[right_child]][0] == self.priority[self.heapq[index]][0]:
                #right child has larger G than parent - swap
                if self.priority[self.heapq[right_child]][1]>self.priority[self.heapq[index]][1]:
                    self.swap(right_child,index)
                    index = right_child
                #right_child G <= parent G
                else:
                    return max_ele
        
      else:#only left child
        
        #parent F < left_child F done
        if self.priority[self.heapq[index]][0]<self.priority[self.heapq[left_child]][0]:
          return max_ele

        #parent F > left_child F need to swap
        elif self.priority[self.heapq[index]][0]>self.priority[self.heapq[left_child]][0]:
            self.swap(left_child,index)
            index = left_child

        #parent F = left_child F check G
        elif self.priority[self.heapq[index]][0] == self.priority[self.heapq[left_child]][0]:
        #parent G >= left_child G done
          if self.priority[self.heapq[index]][1] >= self.priority[self.heapq[left_child]][1]:
            return max_ele
          #parent G < left_child G need to swap
          else: 
            self.swap(index,left_child)
            index = left_child
      
      #incerement child indexes
      left_child = 2*index
      right_child = 2*index+1
      cond1 = right_child<len(self.heapq)
      cond2 = left_child<len(self.heapq)
     

    return max_ele

  def swap(self,child,parent):
    self.heapq[child],self.heapq[parent] = self.heapq[parent],self.heapq[child]

### Queue to generate grids

In [56]:
#queue -> Used to generate random grids with blocked cells
class Queue:
  #Initialization
  def __init__(self):
    self.queue = []
  #checks if queue is empty
  def empty(self):
    return len(self.queue)==0
  #removes the first element and returns it
  def get(self):
    return self.queue.pop(0)
  #adds element to end of queue
  def put(self,x):
    self.queue.append(x)
  #adds element at the beginning
  def putf(self,x):
    self.queue = [x] + self.queue
  #returns the first element
  def peek(self):
    return self.queue[0]


## Grid Generation

In [57]:
import random
#lists to store Grid,start and Goal values
grid_blocked_list = []
grid_start_list = []
grid_goal_list = []

rows = 101
cols= 101
#generating 50 grids
for _ in range(50):
  grid = Grid(101,101,[])
  visited = {}
  #setting all cells to unvisited
  for i in range(grid.rows):
    for j in range(grid.cols):
      visited[(i,j)] = 0

  blocked = {}
  #selecting a random start cell
  start = (random.randint(0,grid.rows-1),random.randint(0,grid.cols-1))
  #selecting a random goal cell
  goal = (random.randint(0,grid.rows-1),random.randint(0,grid.cols-1))
  queue = Queue()

  #starting at the start cell and adding to queue
  queue.put(start)
  reached = {}

  reached[start] = None

  #runs till all cells are visited
  def func():
    while(not queue.empty()):
      current = queue.get()
      visited[current] = 1
      #Visits the neighbors of current cell 
      neighbors = list(grid.neighbors(current))
      random.shuffle(neighbors)
      #selects a random neighbor
      for next in neighbors:
        #With probablity of 30% sets it as blocked
        randomnumber = random.randint(1,10)
        if randomnumber in range(1,4):
          blocked[next] = 1
         #With probablity of 70% sets it as unblocked
        else:
          blocked[next] = 0
        #if its neighbor is not visited add it to the queue 
        if next not in reached and blocked[next] == 0:
          queue.putf(next)
          reached[next] = current
  #if it encounters a dead end then backtrack and add the neigbhor of last visited cell
  for node in visited:
    if visited[node] == 0:
      queue.putf(node)
      func()
  #Setting start and goal as unblocked
  blocked[start]=0
  blocked[goal]=0
  blocked_list = []
  #storing all the blocked,start and goal states
  for node in blocked:
      if blocked[node] == 1:
          blocked_list.append(node)
  grid_blocked_list.append(blocked_list)
  grid_start_list.append(start)
  grid_goal_list.append(goal)

In [58]:
len(grid_blocked_list)

50

In [59]:
rows = 101
cols = 101

In [60]:
import pickle
with open("Grid_blocked_list", "rb") as fp:   # Unpickling
  grid_blocked_list = pickle.load(fp)

with open("Grid_start_list", "rb") as fp:   # Unpickling
  grid_start_list = pickle.load(fp)

with open("Grid_goal_list", "rb") as fp:   # Unpickling
  grid_goal_list = pickle.load(fp)


In [61]:
#Heuristic Function ->Manhattan distance between current and end
def heuristic(current,goal):
  return abs(goal[0]-current[0])+abs(goal[1]-current[1])

In [62]:
#Using Predefined Heapq
import heapq
class PriorityQueueMinGPredfined:
  #Initalization
  def __init__(self):
    self.queue = []
  #checks if empty
  def empty(self):
    return len(self.queue)==0
  #removes and returns top element
  def get(self):
    return heapq.heappop(self.queue)[1]
  #adds element to queue based on priority
  def put(self,x,priority):
    return heapq.heappush(self.queue,(priority,x))
  #returns element with most priority
  def peek(self):
    return self.queue[0][1]

# Repeated Forward with Min G with Predefined Heap

In [63]:
#Search Loop(Uses A*)
def ComputePath():
  
  while(g[goal]>g[Open.peek()]):
    #Takes element with most priority
    current = Open.get()
    #adds it to closed list
    closed.append(current)
    next_list = []
    #adds neighbors which are not in closed to next list
    for node in grid.neighbors(current):
      if node not in closed:
        next_list.append(node)
    #Updates G-value and f-values of neighbors 
    for next in next_list:
      if search[next]<counter:
        g[next] = float('inf')
        search[next] = counter
      if g[next] > g[current] + grid.cost(current,next):
        g[next] = g[current] + grid.cost(current,next)
        came_from[next] = current
        f_value = g[next]+heuristic(next,goal)
        #Add to priority queue after updating priorities
        Open.put(next,(f_value,g[next]))
    #if goal cant be reached break
    if Open.empty():
      break


import timeit
#lists to store time,number of visited cells and number of seraches for comparison
time_list= []
visited_cells= []
num_searches = []
Reched_target_forward_min_G = []
#iterating for 50 grids
for iteration in range(50):
  #calculating time
  start_time = timeit.default_timer()
  start = grid_start_list[iteration]
  goal = grid_goal_list[iteration]
  grid = Grid(101,101,grid_blocked_list[iteration])
  counter = 0
  search = {}
  visited_cells_counter = 0
  #Setting all cells as Unvisited
  for i in range(rows):
    for j in range(cols):
      search[(i,j)] = 0
  #loop till you reach goal or no path is found
  while start!=goal:
    g = {}
    h = {}
    came_from = {}
    counter = counter+1
    #Initializing g for start and goal and putting start in priority Queue
    g[start] = 0
    search[start] = counter
    g[goal] = float('inf')
    search[goal] = counter

    #Setting priority Queue
    Open = PriorityQueueMinGPredfined()
    closed = []
    #checking neighbors of start
    for node in grid.neighbors(start):
      if node in grid.Uniblocked:     
        grid.currblocked.append(node)
    f_value = g[start]+heuristic(start,goal)
    Open.put(start,(f_value,g[start]))
    #Calls A* algorithm
    ComputePath();
    flag = 0
    #flag to check if goal is reachable
    if Open.empty():
      flag==1
      print('I cannot reach the Target');
      break;
    #visited cells counter
    visited_cells_counter += len(closed)
    temp = goal
    path = []
    #Reverse path as we get path in reverse order form A*
    while(temp!=start):
      path.append(temp)
      temp = came_from[temp]
    path.append(start)
    path.reverse()
    #Traverese on the path till you reach goal or a blocked cell
    for node in path:
      if node in grid.Uniblocked:
          break
    start = came_from[node]
    #if goal is reached break
    if node==goal:
      Reched_target_forward_min_G.append(node)
      break
  if flag==0:
    print() 
    visited_cells.append(visited_cells_counter)
    num_searches.append(counter)   
    print('reached target')
    stop_time = timeit.default_timer()
    time_list.append(stop_time - start_time)
    print('iteration '+str(iteration)+'  Time: '+ str(stop_time - start_time)+'  num cells expanded: '+str(visited_cells_counter)+'  num_searches: '+str(counter))  


reached target
iteration 0  Time: 3.9331893749999836  num cells expanded: 72144  num_searches: 93

reached target
iteration 1  Time: 0.12360899999998765  num cells expanded: 5010  num_searches: 17

reached target
iteration 2  Time: 4.241988666999987  num cells expanded: 70615  num_searches: 126

reached target
iteration 3  Time: 0.7624441250000018  num cells expanded: 16399  num_searches: 27

reached target
iteration 4  Time: 5.5410348340000155  num cells expanded: 65965  num_searches: 66

reached target
iteration 5  Time: 0.2866665840000451  num cells expanded: 9714  num_searches: 52

reached target
iteration 6  Time: 0.04870375000007243  num cells expanded: 2947  num_searches: 17

reached target
iteration 7  Time: 0.008242249999966589  num cells expanded: 348  num_searches: 12

reached target
iteration 8  Time: 0.4956248749999759  num cells expanded: 16037  num_searches: 35

reached target
iteration 9  Time: 0.9541938329999766  num cells expanded: 29437  num_searches: 67
I cannot re

In [64]:
sum(time_list)/50

1.3443949175199963

In [65]:
sum(num_searches)/50

40.86

In [66]:
sum(visited_cells)/50

20574.56

# Repeated Forward with Min G with Defined Heap

In [67]:
#Search Loop(Uses A*)
def ComputePath():
  
  while(g[goal]>g[Open.peek()]):
    #Takes element with most priority
    current = Open.get()
    #adds it to closed list
    closed.append(current)
    next_list = []
    #adds neighbors which are not in closed to next list
    for node in grid.neighbors(current):
      if node not in closed:
        next_list.append(node)
    #Updates G-value and f-values of neighbors 
    for next in next_list:
      if search[next]<counter:
        g[next] = float('inf')
        search[next] = counter
      if g[next] > g[current] + grid.cost(current,next):
        g[next] = g[current] + grid.cost(current,next)
        came_from[next] = current
        f_value = g[next]+heuristic(next,goal)
        #Add to priority queue after updating priorities
        Open.put(next,(f_value,g[next]))
    #if goal cant be reached break
    if Open.empty():
      break


import timeit
#lists to store time,number of visited cells and number of seraches for comparison
time_list= []
visited_cells= []
num_searches = []
Reched_target_forward_min_G = []
#iterating for 50 grids
for iteration in range(50):
  #calculating time
  start_time = timeit.default_timer()
  start = grid_start_list[iteration]
  goal = grid_goal_list[iteration]
  grid = Grid(101,101,grid_blocked_list[iteration])
  counter = 0
  search = {}
  visited_cells_counter = 0
  #Setting all cells as Unvisited
  for i in range(rows):
    for j in range(cols):
      search[(i,j)] = 0
  #loop till you reach goal or no path is found
  while start!=goal:
    g = {}
    h = {}
    came_from = {}
    counter = counter+1
    #Initializing g for start and goal and putting start in priority Queue
    g[start] = 0
    search[start] = counter
    g[goal] = float('inf')
    search[goal] = counter

    #Setting priority Queue
    Open = PriorityQueueMinG()
    closed = []
    #checking neighbors of start
    for node in grid.neighbors(start):
      if node in grid.Uniblocked:     
        grid.currblocked.append(node)
    f_value = g[start]+heuristic(start,goal)
    Open.put(start,(f_value,g[start]))
    #Calls A* algorithm
    ComputePath();
    flag = 0
    #flag to check if goal is reachable
    if Open.empty():
      flag==1
      print('I cannot reach the Target');
      break;
    #visited cells counter
    visited_cells_counter += len(closed)
    temp = goal
    path = []
    #Reverse path as we get path in reverse order form A*
    while(temp!=start):
      path.append(temp)
      temp = came_from[temp]
    path.append(start)
    path.reverse()
    #Traverese on the path till you reach goal or a blocked cell
    for node in path:
      if node in grid.Uniblocked:
          break
    start = came_from[node]
    #if goal is reached break
    if node==goal:
      Reched_target_forward_min_G.append(node)
      break
  if flag==0:
    print() 
    visited_cells.append(visited_cells_counter)
    num_searches.append(counter)   
    print('reached target')
    stop_time = timeit.default_timer()
    time_list.append(stop_time - start_time)
    print('iteration '+str(iteration)+'  Time: '+ str(stop_time - start_time)+'  num cells expanded: '+str(visited_cells_counter)+'  num_searches: '+str(counter))  


reached target
iteration 0  Time: 4.9377753329999905  num cells expanded: 70763  num_searches: 89

reached target
iteration 1  Time: 0.13472925000007763  num cells expanded: 3842  num_searches: 14

reached target
iteration 2  Time: 5.210418000000004  num cells expanded: 71920  num_searches: 131

reached target
iteration 3  Time: 1.0365943340000285  num cells expanded: 17867  num_searches: 30

reached target
iteration 4  Time: 6.046273749999955  num cells expanded: 61811  num_searches: 68

reached target
iteration 5  Time: 0.7711677080000072  num cells expanded: 17893  num_searches: 68

reached target
iteration 6  Time: 0.06574604100001125  num cells expanded: 2335  num_searches: 18

reached target
iteration 7  Time: 0.00841354199997113  num cells expanded: 244  num_searches: 8

reached target
iteration 8  Time: 0.7649917500000356  num cells expanded: 17037  num_searches: 38

reached target
iteration 9  Time: 0.7165861250000489  num cells expanded: 15207  num_searches: 40
I cannot reac

In [68]:
sum(time_list)/50

1.6122460016200102

In [69]:
sum(num_searches)/50

38.64

In [70]:
sum(visited_cells)/50

20188.18

# Repeated Forward with Max G with defined Heap

In [71]:
#Search Loop(Uses A*)
def ComputePath():
  
  while(g[goal]>g[Open.peek()]):
    #Takes element with most priority
    current = Open.get()
    #adds it to closed list
    closed.append(current)
    next_list = []
    #adds neighbors which are not in closed to next list
    for node in grid.neighbors(current):
      if node not in closed:
        next_list.append(node)
    #Updates G-value and f-values of neighbors 
    for next in next_list:
      if search[next]<counter:
        g[next] = float('inf')
        search[next] = counter
      if g[next] > g[current] + grid.cost(current,next):
        g[next] = g[current] + grid.cost(current,next)
        came_from[next] = current
        f_value = g[next]+heuristic(next,goal)
        #Add to priority queue after updating priorities
        Open.put(next,(f_value,g[next]))
    #if goal cant be reached break
    if Open.empty():
      break


import timeit
#lists to store time,number of visited cells and number of seraches for comparison
time_list= []
visited_cells= []
num_searches = []
Reched_target_forward_min_G = []
#iterating for 50 grids
for iteration in range(50):
  #calculating time
  start_time = timeit.default_timer()
  start = grid_start_list[iteration]
  goal = grid_goal_list[iteration]
  grid = Grid(101,101,grid_blocked_list[iteration])
  counter = 0
  search = {}
  visited_cells_counter = 0
  #Setting all cells as Unvisited
  for i in range(rows):
    for j in range(cols):
      search[(i,j)] = 0
  #loop till you reach goal or no path is found
  while start!=goal:
    g = {}
    h = {}
    came_from = {}
    counter = counter+1
    #Initializing g for start and goal and putting start in priority Queue
    g[start] = 0
    search[start] = counter
    g[goal] = float('inf')
    search[goal] = counter

    #Setting priority Queue
    Open = PriorityQueueMaxG()
    closed = []
    #checking neighbors of start
    for node in grid.neighbors(start):
      if node in grid.Uniblocked:     
        grid.currblocked.append(node)
    f_value = g[start]+heuristic(start,goal)
    Open.put(start,(f_value,g[start]))
    #Calls A* algorithm
    ComputePath();
    flag = 0
    #flag to check if goal is reachable
    if Open.empty():
      flag==1
      print('I cannot reach the Target');
      break;
    #visited cells counter
    visited_cells_counter += len(closed)
    temp = goal
    path = []
    #Reverse path as we get path in reverse order form A*
    while(temp!=start):
      path.append(temp)
      temp = came_from[temp]
    path.append(start)
    path.reverse()
    #Traverese on the path till you reach goal or a blocked cell
    for node in path:
      if node in grid.Uniblocked:
          break
    start = came_from[node]
    #if goal is reached break
    if node==goal:
      Reched_target_forward_min_G.append(node)
      break
  if flag==0:
    print() 
    visited_cells.append(visited_cells_counter)
    num_searches.append(counter)   
    print('reached target')
    stop_time = timeit.default_timer()
    time_list.append(stop_time - start_time)
    print('iteration '+str(iteration)+'  Time: '+ str(stop_time - start_time)+'  num cells expanded: '+str(visited_cells_counter)+'  num_searches: '+str(counter))  


reached target
iteration 0  Time: 0.09465912500002105  num cells expanded: 2368  num_searches: 50

reached target
iteration 1  Time: 0.02265620800005763  num cells expanded: 543  num_searches: 16

reached target
iteration 2  Time: 0.23660454200000913  num cells expanded: 6297  num_searches: 95

reached target
iteration 3  Time: 0.08027512500007106  num cells expanded: 2121  num_searches: 41

reached target
iteration 4  Time: 0.11827512499996828  num cells expanded: 3092  num_searches: 56

reached target
iteration 5  Time: 0.08709612499990271  num cells expanded: 2188  num_searches: 51

reached target
iteration 6  Time: 0.01711908400000084  num cells expanded: 391  num_searches: 17

reached target
iteration 7  Time: 0.005880375000060667  num cells expanded: 87  num_searches: 6

reached target
iteration 8  Time: 0.08749966699997458  num cells expanded: 2438  num_searches: 57

reached target
iteration 9  Time: 0.08517954199999167  num cells expanded: 2346  num_searches: 52
I cannot reach

In [72]:
sum(time_list)/50

0.15919558340000323

In [73]:
sum(num_searches)/50

40.28

In [74]:
sum(visited_cells)/50

1999.36

# Repeated Backward with Max G with defined Heap

In [75]:
#Search Loop(Uses A*)
def ComputePathBackward():
  
  while(g[start]>g[Open.peek()]):
    #Takes element with most priority
    current = Open.get()
   
    closed.append(current)
    next_list = []
    #adds neighbors which are not in closed to next list
    for node in grid.neighbors(current):
      if node not in closed:
        next_list.append(node)
   
    for next in next_list:
      if search[next]<counter:
        g[next] = float('inf')
        search[next] = counter
      if g[next] > g[current] + grid.cost(current,next):
        g[next] = g[current] + grid.cost(current,next)
        came_from[next] = current
        f_value = g[next]+heuristic(next,start)
        Open.put(next,(f_value,g[next]))
    if Open.empty():
        break

import timeit
#lists to store time,number of visited cells and number of seraches for comparison
time_list= []
visited_cells= []
num_searches = []
#iterating for 50 grids
for iteration in range(50):
  #calculating time
  start_time = timeit.default_timer()
  start = grid_start_list[iteration]
  goal = grid_goal_list[iteration]
  grid = Grid(101,101,grid_blocked_list[iteration])
  visited_cells_counter=0
  counter = 0
  search = {}
  #Setting all cells as Unvisited
  for i in range(rows):
    for j in range(cols):
      search[(i,j)] = 0
  #loop till you reach goal or no path is found
  while start!=goal:
    g = {}
    h = {}
    came_from = {}
    counter = counter+1
    g[goal] = 0
    search[goal] = counter
    g[start] = float('inf')
    search[start] = counter
    #Setting priority Queue
    Open = PriorityQueueMaxG()
    closed = []
    for node in grid.neighbors(start):
      if node in grid.Uniblocked:     
        grid.currblocked.append(node)
    f_value = g[goal]+heuristic(start,goal)
    Open.put(goal,(f_value,g[goal]))
    
    ComputePathBackward();
    flag = 0
    if Open.empty():
      flag = 1
      print('I cannot reach the Target');
      break;
    #visited cells counter
    visited_cells_counter += len(closed)

    temp = start
    path = []
    #Reverse path as we get path in reverse order form A*
    while(temp!=goal):
      path.append(temp)
      temp = came_from[temp]
    path.append(goal)
    
    #Traverese on the path till you reach goal or a blocked cell
    for node in path:
        if node in grid.Uniblocked:
            break
    
    bdx = path.index(node)
    start = path[bdx-1]
    
    if node==goal:
        break
  if flag==0:
    print() 
    visited_cells.append(visited_cells_counter)
    num_searches.append(counter)   
    print('reached target')
    stop_time = timeit.default_timer()
    time_list.append(stop_time - start_time)
    print('iteration '+str(iteration)+'  Time: '+ str(stop_time - start_time)+'  num cells expanded: '+str(visited_cells_counter)+'  num_searches: '+str(counter))  


reached target
iteration 0  Time: 0.6252170000000206  num cells expanded: 12392  num_searches: 60

reached target
iteration 1  Time: 0.1258034160000534  num cells expanded: 3792  num_searches: 20

reached target
iteration 2  Time: 8.371379959000024  num cells expanded: 77234  num_searches: 115

reached target
iteration 3  Time: 0.6092049170000564  num cells expanded: 7310  num_searches: 25

reached target
iteration 4  Time: 8.117202249999991  num cells expanded: 64897  num_searches: 70

reached target
iteration 5  Time: 0.9660350419999304  num cells expanded: 17217  num_searches: 62

reached target
iteration 6  Time: 0.09069958300005965  num cells expanded: 2699  num_searches: 21

reached target
iteration 7  Time: 0.008517417000007299  num cells expanded: 219  num_searches: 10

reached target
iteration 8  Time: 1.5402605000000449  num cells expanded: 23138  num_searches: 36

reached target
iteration 9  Time: 0.3197037500000306  num cells expanded: 5597  num_searches: 21
I cannot reach

In [76]:
sum(time_list)/50

1.8279557983400037

In [77]:
sum(visited_cells)/50

17921.9

In [78]:
sum(num_searches)/50

41.02

# Adaptive A* with MaxG

In [79]:
#ComputePath only For first search
def ComputePathfirst():
  #takes the element with highest priority
  while(g[goal]>g[Open.peek()]):
    current = Open.get()
    visited.append(current)
    closed.append(current)
    next_list = []
    #checks if neighbors are in closed list and adds them to 
    for node in grid.neighbors(current):
      if node not in closed:
        next_list.append(node)
   #update G-values and h-values and add them to queue along with priorities
    for next in next_list:
      if search[next]<counter:
        g[next] = float('inf')
        search[next] = counter
      if g[next] > g[current] + grid.cost(current,next):
        g[next] = g[current] + grid.cost(current,next)
        came_from[next] = current
        Open.put(next,(g[next]+hx[next],g[next]))
    #if goal not reachable break
    if Open.empty():
      break
#Compute path for rest of the searches
def ComputePathAdaptive():
  #takes the element with highest priority
  while(g[goal]>g[Open.peek()]):
    current = Open.get()
    visited.append(current)
    closed.append(current)
    next_list = []
    for node in grid.neighbors(current):
      if node not in closed:
        next_list.append(node)
    #update G-values and h-values and add them to queue along with priorities
    for next in next_list:
      if search[next]<counter:
        g[next] = float('inf')
        search[next] = counter
      if g[next] > g[current] + grid.cost(current,next):
        g[next] = g[current] + grid.cost(current,next)
        came_from[next] = current
        Open.put(next,(g[next]+hx[next],g[next]))
    #if goal not reachable break
    if Open.empty():
      break
import timeit
time_list= []
visited_cells= []
num_searches = []
for iteration in range(50):
  start_time = timeit.default_timer()
  start = grid_start_list[iteration]
  goal = grid_goal_list[iteration]
  grid = Grid(101,101,grid_blocked_list[iteration])
  visited_cells_counter = 0
  counter = 0
  search = {}
  hx = {}
  #setting heuristic values for first search
  for i in range(rows):
    for j in range(cols):
      search[(i,j)] = 0
      hx[(i,j)] = abs(goal[0]-i)+abs(goal[1]-j)
  #loop till goal is reached
  while start!=goal:
    g = {}
    h = {}
    came_from = {}
    #setting g-value and h-value of start and goal and adding them to queue
    counter = counter+1
    g[start] = 0
    search[start] = counter
    g[goal] = float('inf')
    search[goal] = counter
    Open = PriorityQueueMaxG()
    closed = []
    #keep track of blocked neighbors at start
    for node in grid.neighbors(start):
      if node in grid.Uniblocked:     
        grid.currblocked.append(node)
    
    
    visited = []
    #A* for first search similar to repeated foraward
    if counter == 1:  
      Open.put(start,(g[start]+hx[start],g[start]))
      ComputePathfirst();
    #using different values from 2nd search
    else:
      Open.put(start,g[start]+hx[start])
      ComputePathAdaptive()
    flag = 0    
    #if goal is unreachable
    if Open.empty():
      flag = 1
      print('I cannot reach the Target');
      break;
    #track of number of visited cells
    visited_cells_counter += len(closed)
    temp = goal
    path = []
    #Reversing Path returned by A*
    while(temp!=start):
      path.append(temp)
      temp = came_from[temp]
    path.append(start)
    path.reverse()
    #traverse on path till goal is reached or a blocked cell
    for node in path:
      if node in grid.Uniblocked:
        break
    #length of path of current search
    gsgoal = len(path)-1
    #Updating heuristic values of vistited nodes
    for seennode in visited:
        hx[seennode] = gsgoal - g[seennode]
    start = came_from[node]
    #break if goal is reached
    if node==goal:
        break
  if flag==0:
    print() 
    visited_cells.append(visited_cells_counter)
    num_searches.append(counter)   
    print('reached target')
    stop_time = timeit.default_timer()
    time_list.append(stop_time - start_time)
    print('iteration '+str(iteration)+'  Time: '+ str(stop_time - start_time)+'  num cells expanded: '+str(visited_cells_counter)+'  num_searches: '+str(counter))  


reached target
iteration 0  Time: 0.10164050000003044  num cells expanded: 2565  num_searches: 53

reached target
iteration 1  Time: 0.02437162499995793  num cells expanded: 541  num_searches: 16

reached target
iteration 2  Time: 0.20225216699998327  num cells expanded: 5470  num_searches: 91

reached target
iteration 3  Time: 0.07868762500004323  num cells expanded: 2054  num_searches: 40

reached target
iteration 4  Time: 0.11618512500001543  num cells expanded: 3066  num_searches: 56

reached target
iteration 5  Time: 0.09067520899998271  num cells expanded: 2264  num_searches: 53

reached target
iteration 6  Time: 0.019779209000034825  num cells expanded: 406  num_searches: 18

reached target
iteration 7  Time: 0.008637583000108862  num cells expanded: 83  num_searches: 6

reached target
iteration 8  Time: 0.08448683400001755  num cells expanded: 2329  num_searches: 56

reached target
iteration 9  Time: 0.08570116700002472  num cells expanded: 2259  num_searches: 52
I cannot reac

In [80]:
sum(time_list)/50

0.07101848335999648

In [81]:
sum(visited_cells)/50

1824.18

In [82]:
sum(num_searches)/50

39.34

In [83]:
import pickle
with open("Grid_blocked_list", "wb") as fp:   #Pickling
   pickle.dump(grid_blocked_list, fp)

with open("Grid_start_list", "wb") as fp:   #Pickling
   pickle.dump(grid_start_list, fp)

with open("Grid_goal_list", "wb") as fp:   #Pickling
   pickle.dump(grid_goal_list, fp)